In [1]:
#Left Joins Overnight
import pandas as pd
import os
import configparser
import sys
sys.path.append('/project/Development')
from helpers import general_helpers
import configparser
config = configparser.ConfigParser()
config.read('/project/Development/config.ini')
host = config['DATABASE']['HOST']
username = config['DATABASE']['USERNAME']
password = config['DATABASE']['PASSWORD']
new_database = config['DATABASE']['NEW_DB']
old_database = config['DATABASE']['OLD_DB']
temporary_upload = config['DATABASE']['TEMP_UPLOAD_DB']
previous_qa_loc = config['FOLDERS']['OLD_QA_LOC']
new_qa_loc = config['FOLDERS']['NEW_QA_LOC']
latest_expected_date = config['CONSTANTS']['LATEST_DATE']

engine = general_helpers.connect_to_db(host, username, password, new_database)

In [2]:
#Left Joins Overnight
import pandas as pd
import os
import configparser
import sys
sys.path.append('/project/Development')
from helpers import general_helpers
import configparser
config = configparser.ConfigParser()
config.read('/project/Development/config.ini')
host = config['DATABASE']['HOST']
username = config['DATABASE']['USERNAME']
password = config['DATABASE']['PASSWORD']
new_database = config['DATABASE']['NEW_DB']
old_database = config['DATABASE']['OLD_DB']
temporary_upload = config['DATABASE']['TEMP_UPLOAD_DB']
previous_qa_loc = config['FOLDERS']['OLD_QA_LOC']
new_qa_loc = config['FOLDERS']['NEW_QA_LOC']
latest_expected_date = config['CONSTANTS']['LATEST_DATE']

engine = general_helpers.connect_to_db(host, username, password, new_database)

data_list_overnight=[(['rawinventor', 'uuid'], ['persistent_inventor_disambig', 'rawinventor_id']),(['patent', 'id'], ['rawinventor', 'patent_id']), (['inventor', 'id'], ['rawinventor', 'inventor_id']), (['patent', 'id'], ['rawlawyer', 'patent_id'])] 
def left_joins_overnight(new_database, previous_qa_loc, new_qa_loc):
    '''new_db: new database
   previous_qa_loc: location of previous qa document
   new_qa_loc: location of new qa document
   this function does left joins that take a long time 
   in the new database and writes output to a csv file'''
    results = pd.read_excel("{0}/4_left_joins_overnight.xlsx".format(previous_qa_loc))
    left_joins = []
    id_list=[]
    for i, j in data_list_overnight:
        try:
            conn=engine.connect()
            var = conn.execute("select count(distinct {2}.{3}) from {0} right join {2} on {2}.{3} = {0}.{1} where {0}.{1} is null".format(i[0], i[1], j[0], j[1]))
            count = [row[0] for row in var][0]
            print (count)
            left_joins.append("There are {0} rows in {2} not in {1}".format(count, i[0], j[0]))  
            if count >0: # if there are tables in the right table not in the left
                print (count)
                var_ids = pd.read_sql("select distinct {2}.{3} from {0} right join {2} on {2}.{3} = {0}.{1} where {0}.{1} is null limit 5".format(i[0], i[1], j[0], j[1]), conn)
                #print(var_ids)
                ids_sql=[row[0] for row in var_ids][0]
                ids= ids_sql.values.T.tolist()
                id_list.append(ids[0])
            else:
                id_list.append('none')
        except:
            print ("Problem with join: {} and {}".format(i[0],j[0]))
            left_joins.append("Problem with join: {} and {}".format(i[0],j[0]))
            id_list.append('Problem')
    conn.close()   
    results['Description_{0}'.format(new_database)]= left_joins
    results['Example missing IDS_{0}'.format(new_database)]= id_list
    return results    
def write_overnight(results, new_qa_loc):
    results.to_csv("{0}/08_left_joins_overnight.csv".format(new_qa_loc), index = False)
    
overnight_results = left_joins_overnight(new_database, previous_qa_loc, new_qa_loc)
write_overnight(overnight_results, new_qa_loc)

Problem with join: rawinventor and persistent_inventor_disambig
0
0
0


In [11]:
data_list_overnight=[(['rawinventor', 'uuid'], ['persistent_inventor_disambig', 'rawinventor_id']),(['patent', 'id'], ['rawinventor', 'patent_id']), (['inventor', 'id'], ['rawinventor', 'inventor_id']), (['patent', 'id'], ['rawlawyer', 'patent_id'])] 
def left_joins_overnight(new_database, previous_qa_loc, new_qa_loc):
    '''new_db: new database
   previous_qa_loc: location of previous qa document
   new_qa_loc: location of new qa document
   this function does left joins that take a long time 
   in the new database and writes output to a csv file'''
    results = pd.read_excel("{0}/4_left_joins_overnight.xlsx".format(previous_qa_loc))
    left_joins = []
    id_list=[]
    for i, j in data_list_overnight:
        try:
            conn=engine.connect()
            var = conn.execute("select count(distinct {2}.{3}) from {0} right join {2} on {2}.{3} = {0}.{1} where {0}.{1} is null".format(i[0], i[1], j[0], j[1]))
            count = [row[0] for row in var][0]
            left_joins.append("There are {0} rows in {2} not in {1}".format(count, i[0], j[0]))  
            if count >0: # if there are tables in the right table not in the left
                var_ids = conn.execute("select distinct {2}.{3} from {0} right join {2} on {2}.{3} = {0}.{1} where {0}.{1} is null limit 5".format(i[0], i[1], j[0], j[1])
                ids_sql=[row[0] for row in var_ids][0]
                ids= ids_sql.values.T.tolist()
                id_list.append(ids[0])
            else:
                id_list.append('none')
        except:
            print "Problem with join: {} and {}".format(i[0],j[0])
            left_joins.append("Problem with join: {} and {}".format(i[0],j[0]))
            id_list.append('Problem')
    conn.close()        
    results['Description_{0}'.format(new_database)]= left_joins
    results['Example missing IDS_{0}'.format(new_database)]= id_list
    return results
def write_overnight(results, new_qa_loc):
    results.to_csv("{0}/08_left_joins_overnight".format(new_qa_loc), index = False)

SyntaxError: invalid syntax (<ipython-input-11-a04aad914510>, line 19)

In [ ]:
overnight_results = left_joins_overnight(new_db, previous_qa_loc, new_qa_loc)
write_overnight(overnight_results, new_qa_loc)